In [24]:
import pandas as pd
import cv2
import opencv_jupyter_ui as jcv2
from train_model import train_model
from feat import Detector
import warnings
import time
from time import sleep
from furhat_remote_api import FurhatRemoteAPI
import speech_recognition as sr
warnings.filterwarnings(action="ignore", category=FutureWarning)


In [25]:
model = train_model()
detector = Detector(device="cuda")

Train size: 903 Val size: 258 Test size: 129
Validation accuracy: 64.34 %, with 17 neighbors
Test accuracy: 62.79 %, with 17 neighbors


In [41]:
# Furhat IP address
FURHAT_IP = "127.0.1.1"

# Connect to Furhat
furhat = FurhatRemoteAPI(FURHAT_IP)
furhat.set_led(red=100, green=50, blue=50)

# Furhat faces and voices
FACES = {'Bartender': 'Brooklyn'}
VOICES_EN = {'Bartender': 'GregoryNeural'}

# Furhat speech
def bsay(line):
    furhat.say(text=line, blocking=True)
    sleep(1) 

emotion_detection_active = False

# Function to start emotion detection
def start_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = True

# Function to stop emotion detection
def stop_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = False
    
# Speech recognition setup
recognizer = sr.Recognizer()

# Function to recognize speech
def recognize_speech():
    with sr.Microphone() as source:
        print("Say something!")
        audio = recognizer.listen(source, timeout=5)
    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio.")
        return None

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)

last_detection_time = time.time()
emotion = None
while True:
    ret, frame = cam.read()
    if not ret:
        print("OpenCV found an error reading the next frame.")
        break

    current_time = time.time()  # Use a different variable name
    if current_time - last_detection_time >= 2:
        faces = detector.detect_faces(frame)
        landmarks = detector.detect_landmarks(frame, faces)    
        action_units = detector.detect_aus(frame, landmarks)
        action_units = action_units[0]
    
        # Predict emotion
        emotion = model.predict(action_units)
        if emotion == "happiness":
                bsay("You are happy")
                furhat.gesture(name="Smile")
        elif emotion == "sadness":
                bsay("You are sad")
                furhat.gesture(name='Oh')
        elif emotion == "neutral":
                bsay("You are neutral")
                furhat.gesture(name='Wink')
        
        last_detection_time = current_time
    if emotion is not None:
        cv2.putText(frame, emotion[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    jcv2.imshow("Emotion Detection", frame)

    key = jcv2.waitKey(1) & 0xFF
    if key == 27: # ESC pressed
        break

cam.release()
jcv2.destroyAllWindows()

c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to

Exection is stopped


In [ ]:
cam.release()